In [1]:
pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 13.1 MB/s eta 0:00:01     |████████████████████████████▌   | 798 kB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 2.5 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


In [4]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


Downloading: 100%|██████████| 929/929 [00:00<00:00, 1.03MB/s]
Downloading: 100%|██████████| 878k/878k [00:00<00:00, 16.1MB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 6.62MB/s]
Downloading: 100%|██████████| 239/239 [00:00<00:00, 113kB/s]
Downloading: 100%|██████████| 478M/478M [00:19<00:00, 26.2MB/s] 
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

In [5]:
def run_model(text):

  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  # Print labels and scores
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  return scores


In [15]:
import pandas as pd
def extract_from_file(filename):
  df = pd.read_csv(filename)
  df[['negative','neutral','positive']] =  df.apply(lambda x: run_model(x["tweet"]), axis=1, result_type="expand")
  df.to_csv(filename)
  print(df)

In [16]:
extract_from_file("tesla_data/20200607.csv")

                     id      conversation_id               created_at  \
0   1269777156990779393  1269777156990779393  2020-06-07 23:44:00 UTC   
1   1269767173272154114  1269767173272154114  2020-06-07 23:04:19 UTC   
2   1269750661970788360  1269750661970788360  2020-06-07 21:58:43 UTC   
3   1269741592576798720  1269741592576798720  2020-06-07 21:22:40 UTC   
4   1269740642084806657  1269740642084806657  2020-06-07 21:18:54 UTC   
..                  ...                  ...                      ...   
56  1269433967759032331  1269433967759032331  2020-06-07 01:00:17 UTC   
57  1269432657034739713  1269394520443162625  2020-06-07 00:55:05 UTC   
58  1269432633827549184  1269432633827549184  2020-06-07 00:54:59 UTC   
59  1269431284184555521  1269431284184555521  2020-06-07 00:49:37 UTC   
60  1269426171801825282  1269397521916280833  2020-06-07 00:29:18 UTC   

          date      time  timezone              user_id         username  \
0   2020-06-07  23:44:00         0   7486111681

In [9]:
pwd

'/mnt/c/Users/Jacky/My Drive/Spring 22/machine learning/cs475_project'